In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import boto3
import time
from datetime import datetime, timedelta
import pytz

In [6]:
def list_filenames(bucket_name, directory_prefix):
    # Create a boto3 client for S3
    s3 = boto3.client('s3')

    # Initialize a paginator to handle multiple pages of S3 objects (if necessary)
    paginator = s3.get_paginator('list_objects_v2')

    # Create a list to hold the filenames
    filenames = []

    # Use the paginator to fetch all objects within the specified directory
    for page in paginator.paginate(Bucket=bucket_name, Prefix=directory_prefix):
        # Access the 'Contents' from each page which lists the objects
        for obj in page.get('Contents', []):  # Check if 'Contents' key is in page
            filenames.append(obj['Key'])

    return filenames
#Usage
bucket_name = 'moneygans-data'  # Replace with your S3 bucket name
directory_prefix = 'basketball_nba/starting_money/'
#2024-11-01/live_odds/'
# 'baseball_mlb/aggregated_data/2024-07-24_all/'  # Ensure this ends with a slash if you're specifying a folder

# Get the list of filenames
file_list = list_filenames(bucket_name, directory_prefix)
file_list[::2]
# df = pd.read_parquet(f's3://moneygans-data/{file_list[1]}', engine='pyarrow')

['basketball_nba/starting_money/2024-11-01_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-02_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-03_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-04_starting/data/starting_moneylines.parquet']

In [ ]:
def combine_opening_money(df_all_path, df_new):
    df_all = pd.read_parquet(df_all_path, engine='pyarrow')
    df_new_combined = pd.concat([df_all, df_new], axis=0)
    return df_new_combined

df = pd.DataFrame()
for file in file_list[2::2]:
    df_new= pd.read_parquet(f's3://moneygans-data/{file}', engine='pyarrow')
    df_new['date'] = df_new['commence_time_pst'].apply(lambda x: x.split('T')[0]).unique()[0]
    df = pd.concat([df, df_new], axis=0)

    

In [91]:
OPENING_PATH = f's3://moneygans-data/basketball_nba/starting_money/starting_money_all.parquet'
df.to_parquet(OPENING_PATH, engine='pyarrow')


In [ ]:
def combine_opening_money(df_new):
    OPENING_PATH = f's3://moneygans-data/basketball_nba/starting_money/starting_money_all.parquet'
    df_all = pd.read_parquet(OPENING_PATH, engine='pyarrow')
    df_new['date'] = df_new['commence_time_pst'].apply(lambda x: x.split('T')[0]).unique()[0]
    df_new_combined = pd.concat([df_all, df_new], axis=0)
    df_new_combined.to_parquet(OPENING_PATH, engine='pyarrow')